# Practices(1): 정규 표현 메타 기호
---
All rights reserved, 2021-2023 by *Youn-Sik Hong*. 수업 목적으로만 활용 가능.

# assignment
- 이 handout sheet의 마지막 셀에 3개의 문제가 있습니다. 
    - 난이도는 중(medium)입니다.
        - 비어있는 셀에 문제에 대한 정답 코드를 작성하고 실행 결과를 보여야 합니다.
    - 이 문제를 잘 해결하고 **정규표현(4).ipynb**의 난이도 상(high)에 도전해 보세요.
- 모든 셀의 실행 결과가 포함되어 있어야 합니다. 
    - 즉, 모든 셀에는 셀 번호가 있어야 합니다.

In [1]:
import re

정규표현 테스트를 쉽게 할 수 있도록 함수 **text_search()** 를 정의합니다.

In [2]:
def text_search(patterns, text):
    m = re.search(patterns, text)
    if m:
        print(m.group(0)) 
    else:
        print('Not found!')

메타기호 \? 는 패턴이 없거나 한 번 발생(zero or one occurrence)을 의미합니다. 
- 패턴이 없다는 것은 empty string(빈 문자열, $\lambda$ 또는 $\epsilon$)을 의미합니다. 
    - $\lambda$ 는 길이가 0인 문자열입니다. 
- 문자열 연산에서 $\lambda$ 는 곱셈의 1과 같습니다. 
    - 곱셈에서 1을 곱해도 전혀 값이 변하지 않는 것과 같습니다. 

In [3]:
# ab? 패턴에 해당하는 문자열: a, ab
#text_search("ab?", "ac")  
text_search(r"ab?", "ac")  
text_search(r"ab?", "abc")
text_search(r"ab?", "bbc")

a
ab
Not found!


- 괄호로 묶으면 어떻게 될까요? 
    - **ab?** 패턴에 해당하는 문자열이 [a, ab]라는 것은 **a(b?)** 와 같습니다.
    - 그러면 **(ab)?** 패턴에 해당하는 문자열은 [$\lambda$, ab] 이겠죠. 
    - 실행하면서 확인해 볼까요? **(ab)?** 는 아무 것도 출력을 하지 않네요..
        - 뭔가 이상하군요.

In [4]:
text_search(r"a(b)?", "ac")
text_search(r"(ab)?", "ac")

a



- **text_search** 메소드를 조금 수정하면 금방 그 이유를 알 수 있습니다. 
    - 괄호를 사용했기 때문에 subgroup을 확인할 수 있습니다.
    - **match** 객체가 *None*은 아닙니다. 
        - m.group(1)이 None 인 것은 $\lambda$(empty string)이기 때문입니다.
        - $\lambda$ 이니 보일 리가 없죠...

In [5]:
m = re.search("(ab)?", "ac")
if m:
    print('match is not null'); print(m.group(1)) 
else:
    print('Not found!')

match is not null
None


**정규표현(2).ipynb** 에서 사용한 메타기호를 다룹니다. 이제 전혀 낯설지 않을 겁니다.

- 메타기호 \* (star로 읽음)는 없거나 여러 번 발생(zero or more occurrences)을 의미합니다. 
- 패턴이 없을 수 있다는 점이 \+와 차이점입니다.

In [6]:
#ab*=a(b)* 패턴에 해당하는 문자열: a, ab, abb, abbb, abbb....
text_search(r"ab*", "ac")  
text_search(r"ab*", "abc")
text_search(r"ab*", "abbbbbc")
text_search(r"ab*", "cbc")

a
ab
abbbbb
Not found!


- 메타기호 \+ 는 한 번 또는 여러 번 발생(one or more occurrences)을 의미합니다. 
- 최소한 한 번은 발생해야 합니다.

In [7]:
# ab+ = a(b)+ 패턴에 해당하는 문자열: {ab, abb, abbb, abbbb, ....}
#text_search("ab+", "ac") #r은 생략 가능
text_search(r"ab+", "ac") 
text_search(r"ab+", "abc")
text_search(r"ab+", "abbc")

Not found!
ab
abb


- 메타기호 \{ \} 를 사용하여 패턴의 발생 횟수 또는 범위를 지정할 수 있습니다. 
    - \{3, 5\}는 패턴이 3번, 4번, 또는 5번 발생할 수 있다는 뜻입니다.
    - 최소 3번은 발생해야 한다는 뜻도 포함하고 있습니다.

In [8]:
text_search(r"ab{2}", "abbc") # abb
text_search(r"ab{3,5}", "aabbbbbbc")  # abbb, abbbb, abbbbb
text_search(r"ab{3,5}", "aabbc") 
text_search(r"(ab){3,5}", "aabbc") #ababab. abababab, ababababab
text_search(r"a(b{3,5})", "abbbc") #abbb, abbbb, abbbbb

abb
abbbbb
Not found!
Not found!
abbb


- 여기서 잠깐! 최소 3번 이상 발생은 어떻게 지정할까요?
- 아래 예제 문자열에서 b 의 개수는 모두 6개입니다.

In [9]:
text_search(r"ab{3,5}", "aabbbbbbbc") # 3,4,5번 발생
text_search(r"ab{3,}", "aabbbbbbbc")  # 3번 이상 발생한 패턴 중 길이가 가장 긴 패턴

abbbbb
abbbbbbb


- 메타기호 \^(caret)은 텍스트의 맨 처음을 나타내는 기호입니다. 
- 메타기호 \$(dollar)는 텍스트의 맨 마지막을 나타내는 기호입니다.
- 아래 예제는 a로 시작해서 c로 끝나는 문자열을 찾습니다.

In [10]:
text_search("^a.*c$", "abfc", )  # a...c
text_search("^a.*c$", "abfck", )  # a...k

abfc
Not found!


- \w : Any **alphanumeric** character (equivalent to [a-zA-Z0-9_]). 
- \W : Any **non-alphanumeric** character (equivalent to  [^a-zA-Z0-9_])

- text_search()는 \^ 이 있기 때문에 영문자+숫자로 이루어진 단어를 찾습니다.
- 첫 번째 findall() 역시 \^ 을 사용했기 때문에 조건에 맞는 단어는 하나 뿐입니다.
- 두 번째 findall()은 \^ 이 없기 때문에 영문자+숫자로 이루어진 모든 단어를 찾습니다.
    - 구둣점 기호(, !)와 공백 문자(space)는 제외됩니다.
- 세 번째 findall()은 대문자 W를 사용했기 때문에 구둣점 기호와 공백 문자만 찾습니다.

In [11]:
str = "Tuffy eats pie, Loki eats peas!"
text_search("^\w+", str)
print(re.findall(r'^\w+', str))
print(re.findall(r'\w+', str))
print(re.findall(r'\W+', str))

Tuffy
['Tuffy']
['Tuffy', 'eats', 'pie', 'Loki', 'eats', 'peas']
[' ', ' ', ', ', ' ', ' ', '!']


새로운 메타기호입니다. 사실 설명하지 못한 메타기호가 더 많습니다 -_-

- \s : Any whitespace character (equivalent to [ \t\n\r\f\v]).
    - \t(tab), \n(newline), \r(carriage return) 등은 **escape 문자** 입니다.
        - Tab 키, Enter 키, Space 키에 해당하는 문자를 가리킵니다.
    - [ ]기호(brackets) 의미를 정확히 이해하세요. 
        - [ ]안에 속한 기호 중 **하나**를 가리킵니다. 
    - 아래 예제에서 여러 개의 whitespace 문자를 찾기 때문에, 
        - \s대신 \s+와 같이 메타기호 +를 추가했습니다.

- \S : Any non-whitespace character (equivalent to [^ \t\n\r\f\v])
    - 눈치빠른 학생은 지금쯤 ^(caret) 기호 의미를 파악했을 겁니다. 
    - 그리고, 소문자 메타기호와 대문자 메타기호는 정반대의 패턴을 정의한다는 것도요!!!

In [12]:
str = "Tuffy eats pie, Loki eats peas!"
print('non-whitespace chars=', end='')
text_search(r"\S+", str)

print('whitespace chars=', end='')
text_search(r"\s+", str)

non-whitespace chars=Tuffy
whitespace chars= 


- whitespace chars는 아무 것도 출력되지 않았어요.
- 아래 코드를 실행시켜 보세요.

In [13]:
s1 = re.findall(r'\s+', str)
print('찾은 토큰 개수 = {}, whitespace chars = {}'.format(len(s1), s1))
s2 = re.findall(r'\S+', str)
print('찾은 토큰 개수 = {}, non-whitespace chars = {}'.format(len(s2), s2))

찾은 토큰 개수 = 5, whitespace chars = [' ', ' ', ' ', ' ', ' ']
찾은 토큰 개수 = 6, non-whitespace chars = ['Tuffy', 'eats', 'pie,', 'Loki', 'eats', 'peas!']


한 가지 아니면 2개 메타 기호를 사용한 정규표현 예제가 지루해졌죠. 이제 여러 개 메타 기호를 섞어 쓴 예제를 살펴봅시다.
- \w+와 \S+ 차이점에 주목하기 바랍니다.
- \S+는 구둣점도 포함해서 token을 찾습니다. 

In [14]:
str = "Tuffy? eats^ pie#, Loki+ eats~ peas!"

print(re.findall("\S+", str))
print(re.findall("\S+$", str))
print(re.findall("^\S+", str))

['Tuffy?', 'eats^', 'pie#,', 'Loki+', 'eats~', 'peas!']
['peas!']
['Tuffy?']


#### 자! 이제부터 도전 과제로 넘어갑니다. 
자신의 skill이 어느 정도에 도달했는지 테스트해 볼 수 있는 기회입니다.

#### 문제 1(difficulty=low). 아래 단어에서 모음은 모두 몇 개일까요?

In [15]:
word = 'supercalifragilisticexpialidocious'

In [16]:
# 힌트: 영어 알파벳의 모음은 5개입니다.
print("모음 개수: ", len(re.findall(r"[aeiou]", word)))

모음 개수:  16


#### 문제 2(difficulty=medium). 이메일 주소에서 사용자의 id만 추출하세요.
- 하나의 정규표현을 사용해야 합니다.
- id는 @기호 앞에 있는 부분문자열입니다.

In [17]:
test1 = 'yshong.cse.433@inu.ac.kr'
test2 = 'yshong-345%@gmail.com'
test3 = 'yshong!6789@naver.com'
test4 = 'Hong.Youn.Sik@most.go.kr'
test = [test1, test2, test3, test4]

# 패턴 p를  정의하세요
# 힌트: @기호 앞의 id를 찾으려면 해당 패턴을 괄호로 묶어야 합니다.
pattern = r"([\S]+)@"

for p in test:
    m = re.search(pattern, p)
    if m:
        print('id = ', m.group(1)) 
    else:
        print('Not found!') 

id =  yshong.cse.433
id =  yshong-345%
id =  yshong!6789
id =  Hong.Youn.Sik


#### 문제 3(difficulty=medium). 아래 url에서 년도(year), 월(month), 일(day)을 찾아 보세요.

In [18]:
url= "http://www.telegraph.co.uk/formula-1/2017/10/28/"\
     "mexican-grand-prix-2017-time-does-start-tv-channel-odds-lewis1/2017/05/12/"
url

'http://www.telegraph.co.uk/formula-1/2017/10/28/mexican-grand-prix-2017-time-does-start-tv-channel-odds-lewis1/2017/05/12/'

- 먼저 년도(year)부터 찾아봅시다. 
    - 년도가 17이 아니라 2017, 이렇게 표현되어 있군요. 이게 년도의 패턴이겠죠. 
- 또 finditer() 메소드를 자주 사용하니까 함수로 만들어 놓는게 좋겠죠.

In [19]:
def extract_string(pattern, str):
    for match in re.finditer(pattern, str):
        s = match.start()
        e = match.end()
        print('Found "%s" at %d:%d' % (str[s:e], s, e))

In [20]:
re_year = r'\d{4}'
extract_string(re_year, url)

Found "2017" at 37:41
Found "2017" at 67:71
Found "2017" at 111:115


#### 힌트
- 년도(year)와 월(month)은 2017/10, 2017/05와 같은 패턴입니다.
- 년도, 월, 일이 함께 있는 문자열은 2017/10/28, 2017/05/12와 같은 패턴입니다.    

In [21]:
# 문제 3: 당신의 능력을 보여주세요.
# 패턴 re_fulldate를 정의하세요
re_fulldate = r'\d{4}/\d{2}/\d{2}'

extract_string(re_fulldate, url)

Found "2017/10/28" at 37:47
Found "2017/05/12" at 111:121


Great! 여기까지 온 당신은 장애물을 모두 돌파할 수 있는 모든 준비를 다 갖췄습니다. 이제 Practices(2)(**정규표현(4).ipynb)** 에 준비된 difficulty level=high에 도전해 봅시다.